In [83]:
# TestLinear.py
import torch
from torch.autograd import grad
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from math import ceil, floor
as_strided = np.lib.stride_tricks.as_strided


In [66]:
class Layer:
  def __init__(self): self.layers_name = self.__class__.__name__
  def __call__(self, x): return self.forward(x)
  def forward(self, x): raise NotImplementedError
  def backward(self, output_gradient, LR): raise NotImplementedError

class Activation(Layer):
    def __init__(self, activation, activation_prime):
        self.activation = activation
        self.activation_prime = activation_prime

    def __repr__(self): return self.__class__.__name__
    def __call__(self, input): return self.forward(input)
    def forward(self, input):
        self.input = input # save input for the backward pass
        return self.activation(self.input)

    def backward(self, output_gradient):
        return np.multiply(output_gradient, self.activation_prime(self.input))


# ReLU, LeakyReLU, ELU enable faster and better convergence than sigmoids.
# GELU: Gaussian Error Linear Unit used in most Transformers(GPT-3, BERT): paperswithcode.com/method/gelu
# Hard-Swish: paperswithcode.com/method/hard-swish

def sigmoid(x): return np.reciprocal((1.0+np.exp(-np.clip(x, -100, 2e12))))
def sigmoid_prime(x):
  s = 1.0/( (1.0+np.exp(-np.clip(x, -100, 2e12))) )
  return s * (1 - s) # σ(x)*(1-σ(x))
def relu(x): return np.where(x>= 0, x, 0)
def relu_prime(x): return np.where(x>= 0, 1, 0)
def leaky_relu(x, alpha=0.01): return np.where(x>= 0, x, alpha*x)
def leaky_relu_prime(x, alpha=0.01): return np.where(x>= 0, 1, alpha)
def elu(x, alpha=0.01): return np.where(x>= 0, x, alpha*(np.exp(x)-1))
def elu_prime(x, alpha=0.01): return np.where(x>= 0, 1, alpha*np.exp(x))
def swish(x): return x * np.reciprocal((1.0+np.exp(-x))) # x*σ(x) σ(x)+σ'(x)x : σ(x)+σ(x)*(1-σ(x))*x
def swish_prime(x): s = np.reciprocal((1.0+np.exp(-x))); return s+s*(1-s)*x #σ(x)+σ(x)*(1-σ(x))*x
silu, silu_prime = swish, swish_prime # The SiLU function is also known as the swish function.
def tanh(x): return np.tanh(x) # or 2.0*(σ((2.0 * x)))-1.0
def tanh_prime(x): return 1 - np.tanh(x) ** 2
def gelu(x): return 0.5*x*(1+np.tanh(0.7978845608*(x+0.044715*np.power(x,3)))) # sqrt(2/pi)=0.7978845608
def gelu_prime(x): return NotImplemented#Yet Error
def quick_gelu(x): return x*sigmoid(x*1.702) # faster version but inacurate
def quick_gelu_prime(x): return 1.702*sigmoid_prime(x*1.702)
def hardswish(x): return x*relu(x+3.0)/6.0
def hardswish_prime(x): return 1.0/6.0 *relu(x+3)*(x+1.0)
def softplus(x, limit=20.0, beta=1.0): return (1.0/beta) * np.log(1 + np.exp(x*beta))
def softplus_prime(limit=20, beta=1.0): _s = np.exp(x*beta) ; return (beta*_s)/(1+_s)
def relu6(x): return relu(x)-relu(x-6)
def relu6_prime(x): return relu_prime(x)-relu_prime(x-6)


class Sigmoid(Activation):
  def __init__(self): super().__init__(sigmoid, sigmoid_prime)
class Relu(Activation):
  def __init__(self): super().__init__(relu, relu_prime)
class Relu6(Activation):
  def __init__(self): super().__init__(relu6, relu6_prime)
class LeakyRelu(Activation):
  def __init__(self, alpha=0.01): super().__init__(leaky_relu, leaky_relu_prime)
class Elu(Activation):
  def __init__(self, alpha=0.01): super().__init__(elu, elu_prime)
class Swish(Activation):
  def __init__(self): super().__init__(swish, swish_prime)
class Tanh(Activation):
  def __init__(self): super().__init__(tanh, tanh_prime)
class Gelu(Activation):
  def __init__(self): super().__init__(gelu, gelu_prime)
class QuickGelu(Activation):
  def __init__(self): super().__init__(quick_gelu, quick_gelu_prime)
class Hardswish(Activation):
  def __init__(self): super().__init__(hardswish, hardswish_prime)
class Softplus(Activation):
  def __init__(self, limit=20.0, beta=1.0): super().__init__(softplus, softplus_prime)

In [67]:
# Functional

def mse(pred, y): return ((pred-y)**2).mean() # $\sum 1/m (pred-y)^2$
def mseP(pred, y): return 2*(pred-y)/np.prod(y.shape) # F(G(x))' should be G(x)' * F(G(x))': (pred-y)*2

def _pad(Z: np.ndarray, K: np.ndarray, mode: str="valid") -> np.ndarray:
    """ Check arguments and pad for conv/corr """
    if mode not in ["full", "same", "valid"]: raise ValueError("mode must be one of ['full', 'same', 'valid']")
    if Z.ndim != K.ndim: raise ValueError("Z and K must have the same number of dimensions")
    if Z.size == 0 or K.size == 0: raise ValueError(f"Zero-size arrays not supported in convolutions.")
    ZN,ZC,ZH,ZW = Z.shape
    OutCh,KC,KH,KW = K.shape
    if ZC!=KC: raise ValueError(f"Kernel must have the same number channels as Input, got Z.shape:{Z.shape}, W.shape {K.shape}")
    if mode == 'valid' : padding = ((0,0),(0,0), (0,0), (0,0))
    elif mode == 'same':
        # OH = ZH-KH+1 -> ZH=OH+KH-1
        PadTop, PadBottom = floor((KH-1)/2), ceil((KH-1)/2)
        PadLeft, PadRigh = floor((KW-1)/2), ceil((KW-1)/2)
        padding = ((0,0),(0,0), (PadTop, PadBottom),(PadLeft, PadRigh))
    elif mode == 'full':
        PadTop, PadBottom = KH-1, KH-1 # full-convolution aligns kernel edge with the firs pixel of input, thus K-1
        PadLeft, PadRigh = KW-1, KW-1
        padding = ((0,0),(0,0), (PadTop, PadBottom),(PadLeft, PadRigh))
    if np.array(padding).any(): Z = np.pad(Z, padding)
    return Z, K

def _corr2d(Z, W):
    Z = Z.transpose(0,2,3,1) # NCHW -> NHWC
    W = W.transpose(2,3,1,0) # OIKK -> KKIO

    N,ZH,ZW,C_in = Z.shape
    KH,KW,_,C_out = W.shape
    Ns, ZHs, ZWs, Cs = Z.strides

    inner_dim = KH * KW * C_in # Size of kernel flattened
    A = as_strided(Z, shape = (N, ZH-KH+1, ZW-KW+1, KH, KW, C_in), strides = (Ns, ZHs, ZWs, ZHs, ZWs, Cs)).reshape(-1,inner_dim)
    out = A @ W.reshape(-1, C_out)
    return out.reshape(N,ZH-KH+1,ZW-KW+1,C_out).transpose(0,3,1,2) # NHWC -> NCHW

def conv2d(Z, W, mode:str="valid"): return _corr2d(*_pad(Z, np.flip(W), mode))
def corr2d(Z, W, mode:str="valid"): return _corr2d(*_pad(Z, W, mode))
def corr2d_backward(Z, W, TopGrad,  mode:str="valid"):
    WGrad = corr2d(Z.transpose(1,0,2,3), TopGrad.transpose(1,0,2,3)).transpose(1,0,2,3)
    ZGrad = np.flip(np.rot90(corr2d(TopGrad, W.transpose(1,0,2,3), "full"))).transpose(1,0,2,3)
    return WGrad , ZGrad

In [68]:
def affTrans(Z, W, B): return Z.dot(W.T) + B # W: (outF,inF)
def affTransP(TopGrad, Z, W):
    BGrad = TopGrad.sum(axis=0)
    WGrad = TopGrad.T.dot(Z)
    Zgrad = TopGrad.dot(W)
    return Zgrad, WGrad, BGrad

class Layer:
    """ All layers should Only acccept batch of inputs: NCHW"""
    def __init__(self): self.trainable = True
    def __call__(self, x): return self.forward(x)
    def __repr__(self): return f"{self.layers_name}(Z)"
    def forward(self, input): raise NotImplementedError
    def backward(self, TopGrad): raise NotImplementedError

class Linear(Layer):
    def __init__(self, inF, outF, bias=True):
        self.layers_name = self.__class__.__name__
        self.trainable = True
        # self.output_shape = N, inF, outF # We must know MBS
        lim = 1 / np.sqrt(inF) # Only inF used to calculate the limit, avoid saturation..
        self.weight  = np.random.uniform(-lim, lim, (inF, outF))
        self.bias = np.random.randn(outF) * 0.1 if bias else None
        self.params = [self.weight, self.bias]

    def forward(self, input):
        self.input = input
        return affTrans(self.input,self.weight, self.bias)

    def backward(self, TopGrad):
        Zgrad, WGrad, BGrad = affTransP(TopGrad, self.input, self.weight)
        self.grads = (WGrad, BGrad)
        return Zgrad # Bottom grad

MBS = 2
inF, outF = 6, 4
torch.manual_seed(2)
x = torch.rand(MBS,inF)
y = torch.rand(MBS,outF)

TL = nn.Linear(inF,outF)
tpred = TL(x)
tloss = F.mse_loss(tpred, y)
tloss.backward()


ML = Linear(inF,outF)
ML.weight, ML.bias = TL.weight.detach().numpy(), TL.bias.detach().numpy() # capy params

mpred = ML(x.numpy()) # [MBS,inF] [inF,outF] -> [MBS, outF]
mloss = mse(mpred, y.numpy())

# #Backward
mseGrad = mseP(mpred, y.numpy())
mzgrad = ML.backward(mseGrad)
print((TL.weight.grad.detach().numpy() - ML.grads[0]).sum())
print((TL.bias.grad.detach().numpy() - ML.grads[1]).sum())

-5.9604645e-08
-7.450581e-09


In [69]:
class Conv2d(Layer):
    def __init__(self, inCh, outCh, KS, stride=1, padding=0, dilation=1, groups=1, bias=True):
        if isinstance(KS, int): KS = (KS, KS)
        # if isinstance(stride, int): stride = (stride, stride)
        # if isinstance(padding, int): padding = (padding, padding)
        # if isinstance(dilation, int): dilation = (dilation, dilation)
        # Hout = floor((Hin+2*padding[0]*dilation[0]*(KS[0]−1)−1+1)/stride[0])
        # Wout = floor((Win+2*padding[0]*dilation[1]*(KS[1]−1)−1+1)/stride[1])
        # self.outShape = outCh, Hout, Wout

        self.layers_name = self.__class__.__name__
        self.trainable = True
        self.weight = np.random.randn(outCh, inCh, *KS) # (outCh,inCh,H,W)
        self.bias = np.random.randn(outCh) # Each filter has bias, not each conv window
        self.params = [self.weight, self.bias]

    def forward(self, x):
        self.input = x
        return corr2d(x, self.weight) + self.bias[np.newaxis, :, np.newaxis, np.newaxis]

    def backward(self, TopGrad):
        kernels_gradient, input_gradient = corr2d_backward(self.input, self.weight, TopGrad)
        self.grads = (kernels_gradient, TopGrad.sum(axis=(0, 2, 3)))
        return input_gradient

In [79]:
# Check conv: 
MBS, KS = 1, 3
inCh, outCh = 1, 1
H, W = 4, 4

torch.manual_seed(2)
x = torch.rand(MBS, inCh, H, W)

TC = nn.Conv2d(inCh, outCh, KS)
MC = Conv2d(inCh, outCh, KS)
MC.weight = TC.weight.detach().numpy()
MC.bias = TC.bias.detach().numpy()
tpred = TC(x).detach().numpy()
mpred = MC(x.numpy())
# print((tpred-mpred).mean())

tloss = F.mse_loss(tpred, y)

-1.1874363e-08

In [ ]:
Torch_d_loss_dw = grad(outputs=loss, inputs=w) # compute gradients

In [110]:
MBS, inCh, outCh, H, W = 2, 1, 3, 40,40
Z = torch.rand(MBS,inCh,H,W).to(torch.float64)
W = torch.rand(outCh,inCh,3,3).to(torch.float64)
B = torch.rand(outCh).to(torch.float64)
Z.requires_grad_(), W.requires_grad_(), B.requires_grad_()

TC = F.conv2d(Z, W, B)
TopGrad = torch.ones_like(TC)
TWG = grad(TC, W, TopGrad, retain_graph=True)[0]
TBG = grad(TC, B, TopGrad, retain_graph=True)[0]
TZG = grad(TC, Z, TopGrad, retain_graph=True)[0]

MWG, MZG = corr2d_backward(Z.detach().numpy(), W.detach().numpy(), TopGrad.detach().numpy())
print((TZG.numpy()-MZG).sum())
print((TWG.numpy()-MWG).sum())
# print((TBG.numpy()-MBG).sum())

0.0
0.0


In [115]:
for i in range(10):
    MBS, inCh, outCh, H, W = np.random.randint(1, 10, 5)
    H, W = H+10, W+10
    Z = torch.rand(MBS,inCh,H,W).to(torch.float64)
    W = torch.rand(outCh,inCh,3,3).to(torch.float64)
    B = torch.rand(outCh).to(torch.float64)
    Z.requires_grad_(), W.requires_grad_(), B.requires_grad_()
    
    TC = F.conv2d(Z, W, B)
    TopGrad = torch.ones_like(TC)
    TWG = grad(TC, W, TopGrad, retain_graph=True)[0]
    TBG = grad(TC, B, TopGrad, retain_graph=True)[0]
    TZG = grad(TC, Z, TopGrad, retain_graph=True)[0]
    
    MWG, MZG = corr2d_backward(Z.detach().numpy(), W.detach().numpy(), TopGrad.detach().numpy())
    print((TZG.numpy()-MZG).sum())
    print((TWG.numpy()-MWG).sum())
    # print((TBG.numpy()-MBG).sum())

0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
